In [5]:
import glob
import pandas as pd
import os
from pydx7 import load_patch_from_bulk
from utils import serialize_specs
from pydx7 import load_patch
from utils import render_from_specs

In [2]:
def find_syx_files(directory):
    syx_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(".syx"):
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, directory)
                syx_files.append(relative_path)
    return syx_files

target_directory = "/home/hoyeol/projects/GCT634_final/data/dx7_patches/DX7_YAMAHA"
syx_files = find_syx_files(target_directory)
print(syx_files[0])
print(len(syx_files))

GreyMatter E! Card/7.syx
35


In [ ]:
patch_data = load_patch_from_bulk(os.path.join(target_directory, syx_files[0]), patch_number=10, load_from_sysex=True)
patch_data

{'name': 'Agogobell',
 'modmatrix': array([[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]),
 'outmatrix': array([1, 0, 1, 0, 0, 0]),
 'feedback': np.uint8(1),
 'coarse': array([ 0,  7, 23,  1, 31,  1]),
 'fine': array([24,  4, 55,  0, 48,  0]),
 'detune': array([0, 0, 7, 0, 0, 0]),
 'transpose': np.uint8(12),
 'ol': array([58, 79, 79, 99, 99, 99]),
 'eg_rate': array([[99, 99, 99, 99, 99, 99],
        [49, 40, 22, 30, 92, 40],
        [28, 38, 50, 35, 20, 33],
        [12, 20, 21, 43, 44, 35]]),
 'eg_level': array([[91, 91, 99, 99, 99, 99],
        [82, 82,  0, 92,  0, 92],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0]]),
 'sensitivity': array([0, 0, 3, 3, 3, 2]),
 'has_fixed_freqs': True}

In [4]:
print("specs = " + serialize_specs(patch_data))

specs = {
    'name': 'Agogobell',
    'modmatrix': [
		[0, 1, 0, 0, 0, 0],
		[0, 1, 0, 0, 0, 0],
		[0, 0, 0, 1, 0, 0],
		[0, 0, 0, 0, 1, 1],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0]
    ],
    'outmatrix': [1, 0, 1, 0, 0, 0],
    'feedback': 1,
    'coarse': [0, 7, 23, 1, 31, 1],
    'fine': [24, 4, 55, 0, 48, 0],
    'detune': [0, 0, 7, 0, 0, 0],
    'transpose': 12,
    'ol': [58, 79, 79, 99, 99, 99],
    'eg_rate': [
		[99, 99, 99, 99, 99, 99],
		[49, 40, 22, 30, 92, 40],
		[28, 38, 50, 35, 20, 33],
		[12, 20, 21, 43, 44, 35]
    ],
    'eg_level': [
		[91, 91, 99, 99, 99, 99],
		[82, 82, 0, 92, 0, 92],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0]
    ],
    'sensitivity': [0, 0, 3, 3, 3, 2],
    'has_fixed_freqs': True,
}


In [5]:
def load_patch_from_bulk(bulk_patches,patch_number:int = 0,load_from_sysex=False):
    '''
    TODO: Incorporate:  
        - KB RATE Scaling (from dx7 users manual: "The EG for each operator can be set for a
                        long bass decay and a short treble decay - as in an acoustic piano")
        - OP Detune parameter
    
    Args:
        patch_file: Path to dx7 cart file
        patch_number: Position of patch within cart
        load_from_sysex: Set it to 'True' when cart file is a sysex dump
    '''

    patch_offset = 6 if load_from_sysex==True else 0
    for i in [patch_number]:
        patch = bulk_patches[patch_offset + i*128:patch_offset+ (i+1)*128]

    return load_patch(patch)

In [6]:
from tqdm import tqdm
import numpy as np
target_directory = "/home/hoyeol/projects/GCT634_final/data/dx7_patches/DX7_YAMAHA"
data = {
    'patch_number': [],
    'name': [],
    'patch_data': [],
    'file_path': [],
    'syx_file': []
}
syx_files = find_syx_files(target_directory)
total = 0
for j in tqdm(range(len(syx_files))):
    syx_file = syx_files[j]
    bulk_patches = np.fromfile(os.path.join(target_directory, syx_file), dtype=np.uint8)
    num_patches = (bulk_patches.shape[0]-6) // 128
    for i in range(num_patches):
        patch_data = load_patch_from_bulk(bulk_patches, patch_number=i, load_from_sysex=True)
        total += 1
        data['patch_number'].append(i)
        data['patch_data'].append(serialize_specs(patch_data))
        data['file_path'].append('DX7_YAMAHA/' + syx_file)
        data['syx_file'].append(syx_file.split('/')[-1])
        data['name'].append(patch_data['name'])
print("done")
df = pd.DataFrame(data)
df.to_csv('DX7_YAMAHA.csv')
df

  0%|          | 0/35 [00:00<?, ?it/s]/home/hoyeol/projects/GCT634_final/pydx7.py:205: RuntimeWarning: overflow encountered in scalar subtract
  detune[5-op] = f_detune - 7
/home/hoyeol/projects/GCT634_final/pydx7.py:213: RuntimeWarning: overflow encountered in scalar subtract
  transpose = (patch[144]-24)
100%|██████████| 35/35 [00:00<00:00, 340.15it/s]

done


,patch_number,name,patch_data,file_path,syx_file
0,0,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
1,1,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
2,2,Snare Snp,"{\n 'name': 'Snare Snp',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
3,3,Windchime,"{\n 'name': 'Windchime',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
4,4,Crasher 1,"{\n 'name': 'Crasher 1',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
...,...,...,...,...,...
1115,27,BANJO,"{\n 'name': 'BANJO ',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1116,28,HARP 1,"{\n 'name': 'HARP 1',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1117,29,HARP 2,"{\n 'name': 'HARP 2',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1118,30,BASS 3,"{\n 'name': 'BASS 3',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx


In [7]:
from tqdm import tqdm
import numpy as np
target_directory = "/home/hoyeol/projects/GCT634_final/data/dx7_patches/DX7_AllTheWeb"
data = {
    'patch_number': [],
    'name': [],
    'patch_data': [],
    'file_path': [],
    'syx_file': []
}
syx_files = find_syx_files(target_directory)
total = 0
for j in tqdm(range(len(syx_files))):
    syx_file = syx_files[j]
    bulk_patches = np.fromfile(os.path.join(target_directory, syx_file), dtype=np.uint8)
    num_patches = (bulk_patches.shape[0]-6) // 128
    for i in range(num_patches):
        patch_data = load_patch_from_bulk(bulk_patches, patch_number=i, load_from_sysex=True)
        total += 1
        data['patch_number'].append(i)
        data['patch_data'].append(serialize_specs(patch_data))
        data['file_path'].append('DX7_AllTheWeb/' + syx_file)
        data['syx_file'].append(syx_file.split('/')[-1])
        data['name'].append(patch_data['name'])
print("done")
df = pd.DataFrame(data)
df.to_csv('DX7_AllTheWeb.csv')
df

  0%|          | 0/12528 [00:00<?, ?it/s]/home/hoyeol/projects/GCT634_final/pydx7.py:205: RuntimeWarning: overflow encountered in scalar subtract
  detune[5-op] = f_detune - 7
/home/hoyeol/projects/GCT634_final/pydx7.py:213: RuntimeWarning: overflow encountered in scalar subtract
  transpose = (patch[144]-24)
100%|██████████| 12528/12528 [00:36<00:00, 344.46it/s]


done


,patch_number,name,patch_data,file_path,syx_file
0,0,LILPEOPLE,"{\n 'name': 'LILPEOPLE',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
1,1,SPI EFX,"{\n 'name': 'SPI EFX ',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
2,2,CASTENETS,"{\n 'name': 'CASTENETS',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
3,3,CASTENETS,"{\n 'name': 'CASTENETS',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
4,4,DINGL 19.,"{\n 'name': 'DINGL 19.',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
...,...,...,...,...,...
390576,27,SpacedrUm,"{\n 'name': 'SpacedrUm',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TX70.SYX,TX70.SYX
390577,28,RandmNots,"{\n 'name': 'RandmNots',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TX70.SYX,TX70.SYX
390578,29,RndmNotes,"{\n 'name': 'RndmNotes',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TX70.SYX,TX70.SYX
390579,30,RandmNts3,"{\n 'name': 'RandmNts3',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TX70.SYX,TX70.SYX


In [1]:
from scipy.io.wavfile import write
import IPython.display as ipd
import pandas as pd
import numpy as np

df = pd.read_csv('DX7_AllTheWeb.csv')

In [7]:
i = np.random.randint(0,1000)
print(i)

specs = eval(df.iloc[i]['patch_data'])
print(specs)
# audio = render_from_specs(specs, sr=44100, n=64, out_scale=0.75)
# write('audio.wav', 44100, audio)
# ipd.Audio(audio, rate=44100)

725
{'name': 'F Bass  1', 'modmatrix': [[0, 1, 1, 0, 1, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0]], 'outmatrix': [1, 0, 0, 0, 0, 0], 'feedback': 6, 'coarse': [8, 0, 4, 0, 2, 1], 'fine': [0, 0, 0, 0, 0, 0], 'detune': [0, 0, 0, 0, 0, 0], 'transpose': 232, 'ol': [84, 73, 79, 61, 92, 99], 'eg_rate': [[99, 98, 96, 50, 99, 72], [64, 67, 19, 52, 71, 30], [44, 38, 20, 35, 35, 25], [54, 54, 54, 41, 51, 51]], 'eg_level': [[99, 86, 99, 99, 82, 99], [92, 92, 92, 92, 92, 92], [56, 74, 89, 91, 88, 90], [0, 0, 0, 0, 0, 0]], 'sensitivity': [2, 1, 0, 1, 1, 1], 'has_fixed_freqs': False}


In [8]:
audio = render_from_specs(specs, sr=44100, n=64, out_scale=0.75)

In [15]:
write('audio.wav', 44100, audio)

In [9]:
print(df.iloc[i]['patch_data'])

{
    'name': 'F Bass  1',
    'modmatrix': [
		[0, 1, 1, 0, 1, 0],
		[0, 1, 0, 0, 0, 0],
		[0, 0, 0, 1, 0, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 1],
		[0, 0, 0, 0, 0, 0]
    ],
    'outmatrix': [1, 0, 0, 0, 0, 0],
    'feedback': 6,
    'coarse': [8, 0, 4, 0, 2, 1],
    'fine': [0, 0, 0, 0, 0, 0],
    'detune': [0, 0, 0, 0, 0, 0],
    'transpose': 232,
    'ol': [84, 73, 79, 61, 92, 99],
    'eg_rate': [
		[99, 98, 96, 50, 99, 72],
		[64, 67, 19, 52, 71, 30],
		[44, 38, 20, 35, 35, 25],
		[54, 54, 54, 41, 51, 51]
    ],
    'eg_level': [
		[99, 86, 99, 99, 82, 99],
		[92, 92, 92, 92, 92, 92],
		[56, 74, 89, 91, 88, 90],
		[0, 0, 0, 0, 0, 0]
    ],
    'sensitivity': [2, 1, 0, 1, 1, 1],
    'has_fixed_freqs': False,
}


In [14]:
from scipy.io.wavfile import write
import IPython.display as ipd
import pandas as pd
import numpy as np
from tqdm import tqdm

In [47]:
import copy

def _make_hashable_representation(data_item, ignore_keys_set):
    """
    Creates a hashable representation of a dictionary item,
    excluding specified keys and converting lists to tuples.
    """
    # Create a temporary dictionary with only the keys to compare
    temp_dict = {}
    for key, value in data_item.items():
        if key not in ignore_keys_set:
            # Recursively convert lists to tuples to make them hashable
            if isinstance(value, list):
                temp_dict[key] = _convert_lists_to_tuples_recursive(value)
            # Add elif for dicts if dicts can be values and need to be part of the signature
            # elif isinstance(value, dict):
            #     temp_dict[key] = tuple(sorted((k, _convert_lists_to_tuples_recursive(v)) for k, v in value.items()))
            else:
                temp_dict[key] = value
    
    # Sort items by key to ensure consistent order for the hashable representation
    # Then convert to a tuple of (key, value) pairs, making it hashable
    return tuple(sorted(temp_dict.items()))

def _convert_lists_to_tuples_recursive(item):
    """
    Recursively converts lists within a structure to tuples.
    """
    if isinstance(item, list):
        return tuple(_convert_lists_to_tuples_recursive(sub_item) for sub_item in item)
    # Add elif for dicts if they can appear as values inside lists/other structures
    # elif isinstance(item, dict):
    #     return tuple(sorted((k, _convert_lists_to_tuples_recursive(v)) for k, v in item.items()))
    return item

def mark_duplicates(data_list, ignore_keys=('name', 'transpose')):
    """
    Identifies duplicate dictionaries in a list, ignoring specified keys,
    and marks later occurrences with 'duplicate': True.

    Args:
        data_list (list): A list of dictionaries.
        ignore_keys (tuple, optional): A tuple of keys to ignore during comparison.
                                       Defaults to ('name', 'transpose').

    Returns:
        list: The original list with 'duplicate' keys added/updated in place.
    """
    if not isinstance(data_list, list):
        raise TypeError("Input must be a list of dictionaries.")
    if not all(isinstance(item, dict) for item in data_list):
        raise ValueError("All items in the list must be dictionaries.")

    seen_signatures = set()
    ignore_keys_set = set(ignore_keys) # Use a set for faster lookups
    duplicates_idx = []
    for i in tqdm(range(len(data_list))):
        item = data_list[i]
        # Generate a hashable signature for the current item, excluding ignored keys
        signature = _make_hashable_representation(item, ignore_keys_set)

        if signature in seen_signatures:
            duplicates_idx.append(i)
        else:
            # It's the first time we've seen this signature
            seen_signatures.add(signature)
            
    return duplicates_idx

In [42]:
df = pd.read_csv('DX7_YAMAHA.csv')

In [43]:
dup_idxs = mark_duplicates([eval(x) for x in df['patch_data'].tolist()])
print(len(dup_idxs))

100%|██████████| 1120/1120 [00:00<00:00, 39919.61it/s]

58


In [44]:
df = df.drop(dup_idxs, axis=0).reset_index(drop=True).drop('Unnamed: 0', axis=1)
df.to_csv('DX7_YAMAHA_deduplicated.csv')
df

,patch_number,name,patch_data,file_path,syx_file
0,0,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
1,1,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
2,2,Snare Snp,"{\n 'name': 'Snare Snp',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
3,3,Windchime,"{\n 'name': 'Windchime',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
4,4,Crasher 1,"{\n 'name': 'Crasher 1',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
...,...,...,...,...,...
1057,26,LUTE,"{\n 'name': 'LUTE ',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1058,28,HARP 1,"{\n 'name': 'HARP 1',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1059,29,HARP 2,"{\n 'name': 'HARP 2',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx
1060,30,BASS 3,"{\n 'name': 'BASS 3',\n 'modmatrix': ...",DX7_YAMAHA/Factory patches/rom1b.syx,rom1b.syx


In [49]:
df = pd.read_csv('DX7_AllTheWeb.csv')

In [50]:
dup_idxs = mark_duplicates([eval(x) for x in df['patch_data'].tolist()])
print(len(dup_idxs))

100%|██████████| 390581/390581 [00:08<00:00, 46682.90it/s]

350793


In [51]:
df = df.drop(dup_idxs, axis=0).reset_index(drop=True).drop('Unnamed: 0', axis=1)
df.to_csv('DX7_AllTheWeb_deduplicated.csv')
df

,patch_number,name,patch_data,file_path,syx_file
0,0,LILPEOPLE,"{\n 'name': 'LILPEOPLE',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
1,1,SPI EFX,"{\n 'name': 'SPI EFX ',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
2,2,CASTENETS,"{\n 'name': 'CASTENETS',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
3,4,DINGL 19.,"{\n 'name': 'DINGL 19.',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
4,5,BELLO,"{\n 'name': 'BELLO ',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
...,...,...,...,...,...
39783,26,B/SlvrMn,"{\n 'name': 'B/SlvrMn ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
39784,27,Bs/NylGtr,"{\n 'name': 'Bs/NylGtr',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
39785,29,BsE.Tack,"{\n 'name': 'BsE.Tack ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TOP40A.SYX,TOP40A.SYX
39786,22,B/SqrLead,"{\n 'name': 'B/SqrLead',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/RYTHSECB.SYX,RYTHSECB.SYX


In [53]:
df1 = pd.read_csv('DX7_YAMAHA_deduplicated.csv')
df2 = pd.read_csv('DX7_AllTheWeb_deduplicated.csv')
df = pd.concat([df1, df2]).reset_index(drop=True).drop('Unnamed: 0', axis=1)
dup_idxs = mark_duplicates([eval(x) for x in df['patch_data'].tolist()])
print(len(dup_idxs))
assert np.all(np.array(dup_idxs) >= len(df1))
df = df.drop(dup_idxs, axis=0).reset_index(drop=True)
df.to_csv('DX7_combined_deduplicated.csv', index=False)
df

100%|██████████| 40850/40850 [00:01<00:00, 34626.86it/s]


1062


,patch_number,name,patch_data,file_path,syx_file
0,0,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
1,1,Triangle,"{\n 'name': 'Triangle ',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
2,2,Snare Snp,"{\n 'name': 'Snare Snp',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
3,3,Windchime,"{\n 'name': 'Windchime',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
4,4,Crasher 1,"{\n 'name': 'Crasher 1',\n 'modmatrix': ...",DX7_YAMAHA/GreyMatter E! Card/7.syx,7.syx
...,...,...,...,...,...
39783,26,B/SlvrMn,"{\n 'name': 'B/SlvrMn ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
39784,27,Bs/NylGtr,"{\n 'name': 'Bs/NylGtr',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
39785,29,BsE.Tack,"{\n 'name': 'BsE.Tack ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TOP40A.SYX,TOP40A.SYX
39786,22,B/SqrLead,"{\n 'name': 'B/SqrLead',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/RYTHSECB.SYX,RYTHSECB.SYX


In [55]:
df = df[len(df1):].reset_index(drop=True)
df.to_csv('DX7_AllTheWeb_deduplicated.csv', index=False)
df

,patch_number,name,patch_data,file_path,syx_file
0,0,LILPEOPLE,"{\n 'name': 'LILPEOPLE',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
1,1,SPI EFX,"{\n 'name': 'SPI EFX ',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
2,2,CASTENETS,"{\n 'name': 'CASTENETS',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
3,4,DINGL 19.,"{\n 'name': 'DINGL 19.',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
4,5,BELLO,"{\n 'name': 'BELLO ',\n 'modmatrix': ...",DX7_AllTheWeb/Aminet/7.syx,7.syx
...,...,...,...,...,...
38721,26,B/SlvrMn,"{\n 'name': 'B/SlvrMn ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
38722,27,Bs/NylGtr,"{\n 'name': 'Bs/NylGtr',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/SPRSPLTA.SYX,SPRSPLTA.SYX
38723,29,BsE.Tack,"{\n 'name': 'BsE.Tack ',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/TOP40A.SYX,TOP40A.SYX
38724,22,B/SqrLead,"{\n 'name': 'B/SqrLead',\n 'modmatrix': ...",DX7_AllTheWeb/LiVeMuSiC/RYTHSECB.SYX,RYTHSECB.SYX


In [56]:
df = pd.read_csv('data/DX7_YAMAHA_deduplicated.csv')
df['split'] = 'train'
random_idxs = np.random.permutation(len(df))[:100]
df.loc[random_idxs, 'split'] = 'test'
df.to_csv('data/DX7_YAMAHA_deduplicated.csv', index=False)

df = pd.read_csv('data/DX7_AllTheWeb_deduplicated.csv')
df['split'] = 'train'
df.to_csv('data/DX7_AllTheWeb_deduplicated.csv', index=False)